# Network

## Pre processing data

In [1]:
import pymongo
import nltk
import numpy as np
from collections import defaultdict, Counter
from tqdm.notebook import tqdm

In [2]:
client = pymongo.MongoClient('mongodb://localhost:27017')
db = client['vatican']
dataset = db['perturbedDataset']

In [3]:
docs = list(dataset.find())

The first task is to create a list of chars and words used in documents. 

In [4]:
chars = ''
for i, d in tqdm(enumerate(docs)):
    for s in d['sentences']:
        for w in s[0]:
            for c in w: 
                if chars.find(c) == -1:
                    chars += c
                    
CHAR_INDEX = dict((c, i) for i, c in enumerate(chars))

In [5]:
words = []

for i, d in tqdm(enumerate(docs)):
    for s in d['sentences']: 
        for w in s[1]: 
            if w not in words: 
                words.append(w)

Dividing the dataset in training set and test set.

In [6]:
TRAINING_SET_SIZE = 80
TEST_SET_SIZE = 100 - TRAINING_SET_SIZE
TRSH = int((len(docs) / 100) * TRAINING_SET_SIZE)

In [7]:
training_set = docs[:TRSH]
test_set = docs[TRSH:]

In [8]:
def preprocess(s):
    out = ""
    for c in s.lower():
        if c in CHAR_INDEX.keys():
            out += c
        else:
            out += '#'
            
    if(len(s) == 1):
        return '#'
    
    return "".join(out)

In [9]:
wordDataset = dict()

for i, d in tqdm(enumerate(training_set)):
    for j, s in enumerate(d['sentences']):
        for k, w in enumerate(s[0]):
            wordDataset.setdefault(s[1][k], []).append(preprocess(w))            

training_data = []
for wk in tqdm(wordDataset.keys()):
    item = []
    for w in wordDataset.get(wk):
        if(w not in item):
            item.append(w)
    training_data.append((item, wk))

#Potrebbe essere eliminato?
wordOccurrences = []            
for wk in tqdm(wordDataset.keys()):
    item = dict(Counter(wordDataset.get(wk)))
    wordOccurrences.append({'word': preprocess(wk), 'occurrences': item})

## Network Setup 

In [10]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(42)

In [11]:
N_LABELS = len(words)
V = len(CHAR_INDEX) * len(CHAR_INDEX)
LABEL_INDEX = dict((l, i) for i, l in enumerate(words))

In [12]:
def string_to_matrix(s, n=2):
    z = preprocess(s)
    C = np.zeros((len(CHAR_INDEX), len(CHAR_INDEX)))
    
    for a, b in nltk.ngrams(z, n=n):
        C[CHAR_INDEX[a], CHAR_INDEX[b]] += 1
        
    C /= C.max()

    return C

In [13]:
class Simple2Gram(nn.Module):
    def __init__(self, num_labels, size):
        super(Simple2Gram, self).__init__()
        self.linear = nn.Linear(size, num_labels)
        
    def forward(self, vec):
        return F.log_softmax(self.linear(vec), dim = 1)

In [14]:
def vector(s, n=2):
    vec = torch.tensor(string_to_matrix(s, n = n)).float()
    return vec.view(1, -1)

def target(label):
    return torch.LongTensor([LABEL_INDEX[label]])

In [15]:
model = Simple2Gram(N_LABELS, V)

### Training

In [16]:
loss = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.1)

In [ ]:
for epoch in tqdm(range(2)): 
    for samples, label in tqdm(training_data):
        for instance in samples:
            model.zero_grad()
            vec = vector(instance)
            tar = target(label)
            log_probs = model(vec)
            L = loss(log_probs, tar)
            L.backward()
            optimizer.step()

<ipython-input-12-e47ac23ba282>:8: RuntimeWarning: invalid value encountered in true_divide
  C /= C.max()
